In [8]:
%pip install --upgrade --quiet docx2txt langchain-community
%pip install -qU langchain-text-splitters langchain-chroma langchain-core langchain-upstage
%pip install -U langchain langchainhub --quiet
%pip install -qU langchain langchain-pinecone langchain-openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # Chunk 한개가 가질 수 있는 토큰 수
    chunk_overlap=300, # Chunk 를 겹치게 자르는, 1 - 3, 다음은 2 - 4 이런식으로
)

# 문서를 가져와서,
loader = Docx2txtLoader('./tax.docx')

# Chunk 단위로 문서를 쪼갠다
document_list = loader.load_and_split(
    text_splitter=text_splitter
)

In [19]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain_openai import ChatOpenAI

load_dotenv()

llm = ChatUpstage()
#llm = ChatOpenAI(model='gpt-4o')

embedding = UpstageEmbeddings(
    model="solar-embedding-1-large"
)

- Chroma : Vector in-memory Database

In [7]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)
index_name = "tax-upstage-index"

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, # Chunk 한개가 가질 수 있는 토큰 수
    chunk_overlap=100, # Chunk 를 겹치게 자르는, 1 - 3, 다음은 2 - 4 이런식으로
)

chunked_documents = text_splitter.split_documents(document_list)

database = PineconeVectorStore.from_documents(
    documents=[],
    embedding=embedding,
    index_name=index_name
)

batch_size = 100
for i in range(0, len(chunked_documents), batch_size):
    print(f'index: {i}, batch size : {batch_size}')
    batch = chunked_documents[i: i + batch_size]
    database.add_documents(batch)

index: 0, batch size : 100
index: 100, batch size : 100
index: 200, batch size : 100
index: 300, batch size : 100
index: 400, batch size : 100


In [8]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrived_docs = database.similarity_search(query, k = 3)
print(retrived_docs)

[Document(id='e23d84b8-b9c6-4632-862a-517996ab2117', metadata={'source': './tax.docx'}, page_content='⑥ 제4항의 신청이 없는 경우에도 제1항을 적용한다.\n\n⑦ 집단적으로 재해가 발생한 경우에는 대통령령으로 정하는 바에 따라 관할 세무서장이 조사결정한 자산상실비율에 따라 제1항을 적용한다.\n\n⑧ 재해손실세액공제에 관하여 필요한 사항은 대통령령으로 정한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제59조(근로소득세액공제) ①근로소득이 있는 거주자에 대해서는 그 근로소득에 대한 종합소득산출세액에서 다음의 금액을 공제한다. <개정 2014. 1. 1., 2015. 5. 13.>\n\n\n\n② 제1항에도 불구하고 공제세액이 다음 각 호의 구분에 따른 금액을 초과하는 경우에 그 초과하는 금액은 없는 것으로 한다.<신설 2014. 1. 1., 2015. 5. 13., 2022. 12. 31.>\n\n1. 총급여액이 3천 300만원 이하인 경우: 74만원\n\n2. 총급여액이 3천 300만원 초과 7천만원 이하인 경우: 74만원 - [(총급여액 - 3천 300만원) × 8/1000]. 다만, 위 금액이 66만원보다 적은 경우에는 66만원으로 한다.\n\n3. 총급여액이 7천만원 초과 1억2천만원 이하인 경우: 66만원 - [(총급여액 - 7천만원) × 1/2]. 다만, 위 금액이 50만원보다 적은 경우에는 50만원으로 한다.\n\n4. 총급여액이 1억2천만원을 초과하는 경우: 50만원 - [(총급여액 - 1억2천만원) × 1/2]. 다만, 위 금액이 20만원보다 적은 경우에는 20만원으로 한다.\n\n③ 일용근로자의 근로소득에 대해서 제134조제3항에 따른 원천징수를 하는 경우에는 해당 근로소득에 대한 산출세액의 100분의 55에 해당하는 금액을 그 산출세액에서 공제한다.<개정 2014. 1. 1.>\n\n[전문개정 2012. 1. 1.]'), Document(id='

In [12]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [20]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = database.as_retriever(),
    chain_type_kwargs={"prompt" : prompt}
)

ai_message = qa_chain.invoke({"query" : query})
print(ai_message)

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?', 'result': '연봉 5천만원인 직장인의 소득세는 정확히 계산하기 위해서는 공제 항목 등 추가 정보가 필요합니다. 하지만 대략적인 계산을 위해 근로소득공제, 기본공제 등을 적용하면, 소득세는 약 300만원~400만원 정도가 될 수 있습니다. 정확한 금액은 개인의 공제 항목 등에 따라 달라질 수 있으니, 연말정산 내역 등을 확인해보시는 것이 좋습니다.'}


In [22]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
다음 문맥을 바탕으로 질문에 답하세요.
<context>
{context}
</context>

질문: {input}

""")

combine_docs_chain = create_stuff_documents_chain(llm, prompt)

retriever = database.as_retriever()
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

response = rag_chain.invoke({"input": query})
print(response["answer"])

연봉 5천만원의 직장인의 소득세를 계산하기 위해서는 다음과 같은 단계를 거칩니다.

1. 총급여액 계산: 연봉 5천만원이 총급여액이 됩니다.

2. 근로소득공제 적용: 총급여액 5천만원에서 근로소득공제액인 2천만원을 공제합니다.
   \[
   \text{근로소득공제} = 2천만원
   \]
   \[
   \text{근로소득금액} = 5천만원 - 2천만원 = 3천만원
   \]

3. 기본공제 적용: 기본공제는 150만원입니다.
   \[
   \text{기본공제} = 150만원
   \]

4. 소득세 과세표준 계산: 근로소득금액에서 기본공제를 뺀 금액이 소득세 과세표준입니다.
   \[
   \text{소득세 과세표준} = 3천만원 - 150만원 = 2천8백5십만원
   \]

5. 소득세율 적용: 소득세율은 소득세 과세표준에 따라 다르게 적용됩니다. 2천8백5십만원의 경우 15%의 세율이 적용됩니다.
   \[
   \text{산출세액} = 2천8백5십만원 \times 15\% = 4백2십7만5천원
   \]

6. 근로소득세액공제 적용: 근로소득세액공제는 총급여액에 따라 다르게 계산됩니다. 총급여액이 3천3백만원 초과 7천만원 이하인 경우, 공식에 따라 계산합니다.
   \[
   \text{근로소득세액공제} = 74만원 - [(5천만원 - 3천3백만원) \times 8/1000]
   \]
   \[
   = 74만원 - [1천7백만원 \times 8/1000]
   \]
   \[
   = 74만원 - 13만6천원 = 60만4천원
   \]

7. 최종 소득세 계산: 산출세액에서 근로소득세액공제를 뺀 금액이 최종 소득세입니다.
   \[
   \text{최종 소득세} = 4백2십7만5천원 - 60만4천원 = 3백6십3만1천원
   \]

따라서, 연봉 5천만원의 직장인의 소득세는 3백6십3만1천원이 됩니다.
